In [1]:
import tflearn
import numpy as np
import pandas as pd
import tensorflow as tf

In [4]:
with open('./new_data/dataset.txt','r') as f:
    lines = f.read().splitlines()
cleaned = lines[0][2:-2].split('], [') #[2:-2] to remove '[[' and  ']]', then separate each row

In [9]:
data_size = len(cleaned)
test_set_size = int(np.floor(0.2 * data_size))
train_set_size = int(data_size - test_set_size)

In [11]:
TRAIN_DIR = "./new_data/cleaned_train.txt"
TEST_DIR = "./new_data/cleaned_test.txt"

In [12]:
#create new text file with cleaned train data
with open(TRAIN_DIR, 'w') as f:
    for row in cleaned[:train_set_size]:
        f.write("%s\n" % row)

In [13]:
#create new text file with cleaned data
with open(TEST_DIR, 'w') as f:
    for row in cleaned[train_set_size:]:
        f.write("%s\n" % row)

In [53]:
def load_train_data(path):
    df_train = pd.read_csv(path, header=None, error_bad_lines=False)
    df_train.rename(columns={52: "result"}, inplace=True) #rename the last col to results
    df_train.dropna(inplace=True) #drop rows with NaN values
    df_train["opposite_result"] = df_train["result"].map(lambda x: 1 if x == 0 else 0) 
    shuffled = df_train.sample(frac=1)
    X_train = shuffled.iloc[:, :-2].as_matrix()
    Y_train = shuffled[["result", "opposite_result"]].as_matrix()
    return X_train, Y_train

In [54]:
def load_test_data(path):
    df_test = pd.read_csv(path, header=None, error_bad_lines=False)
    df_test.rename(columns={52: "result"}, inplace=True) #rename the last col to results
    df_test.dropna(inplace=True) #drop rows with NaN values
    df_test["opposite_result"] = df_test["result"].map(lambda x: 1 if x == 0 else 0)
    shuffled = df_test.sample(frac=1)
    X_test = shuffled.iloc[:, :-2].as_matrix()
    Y_test = shuffled[["result", "opposite_result"]].as_matrix()
    return X_test, Y_test

In [55]:
# Create neural network model
def create_model(num_features, lr=1E-3, two_fc=True, board_dir="", dropout=0.5, activation="relu"):
    net = tflearn.input_data(shape=[None, num_features])
    trunc_norm = tflearn.initializations.truncated_normal(stddev=1/np.sqrt(num_features))
    net = tflearn.fully_connected(net, 100, weights_init=trunc_norm, activation=activation, name="fc1")
    #two_fc: whether to use 1 hidden layer or 2
    if two_fc:
        #dropout to prevent overfitting
        net = tflearn.dropout(net, dropout)
        net = tflearn.fully_connected(net, 80, activation=activation, weights_init=trunc_norm, name="fc2")
    net = tflearn.fully_connected(net, 2, activation="softmax", weights_init=trunc_norm, name="fc3")
    net = tflearn.regression(net, optimizer='adam', loss='categorical_crossentropy', learning_rate=lr)
    #name of tensorboard file
    tensorboard_dir = "/tmp/tflearn_logs/lol/" + board_dir
    # Define model
    model = tflearn.DNN(net, tensorboard_verbose=2, tensorboard_dir=tensorboard_dir)
    return model

In [56]:
def fit_model(model, x_train, y_train, x_test, y_test, epoch=10):
    # Start training (apply gradient descent algorithm)
    model.fit(x_train, y_train, n_epoch=epoch, validation_set=(x_test, y_test),
            snapshot_step=100, show_metric=True, run_id="lol")

In [57]:
X_train, Y_train = load_train_data(TRAIN_DIR)
X_test, Y_test = load_test_data(TEST_DIR)

b'Skipping line 8071: expected 53 fields, saw 61\n'
b'Skipping line 20160: expected 53 fields, saw 61\n'
b'Skipping line 6908: expected 53 fields, saw 61\n'


In [58]:
# weird bug, have to reset graph every time fit_model run again
tf.reset_default_graph()

In [29]:
# Generate unique file name for tensorboard visualization
def make_hparam_string(learning_rate=1E-3, use_two_fc=True, epoch=5, dropout=0.5):
    layers = "2" if use_two_fc else "1"
    return "lr=" + str(learning_rate) + ",fc=" + layers + ",epochs=" + str(epoch) + ",dropout=" + str(dropout)

In [30]:
# Find the optimal learning rate and number of layers
def find_optimal_lr_and_fc():
    for learn_rate in [1E-3, 1E-4, 1E-5]:
        for use_two_fc in [True, False]:
            param_string = make_hparam_string(learning_rate=learn_rate, use_two_fc=use_two_fc)
            model = create_model(num_features=X_train.shape[1], lr=learn_rate, two_fc=use_two_fc, board_dir=param_string)
            fit_model(model, X_train, Y_train, X_test, Y_test)
            # weird bug, have to reset graph every time fit_model run again
            tf.reset_default_graph()

In [256]:
# find_optimal_lr_and_fc()
# OPTIMAL LR: 1E-3
# OPTIMAL LAYERS: Two hidden layers (0.6664 log loss vs 0.6669 log loss)

In [257]:
# Finds optimal # of epochs
def find_optimal_epochs():
    for epoch in [5, 6, 7, 8, 9, 10]:
        param_string = make_hparam_string(epoch=epoch)
        model = create_model(num_features=X_train.shape[1], board_dir=param_string, dropout=dropout)
        fit_model(model, X_train, Y_train, X_test, Y_test, epoch=epoch)
        # weird bug, have to reset graph every time fit_model run again
        tf.reset_default_graph()

In [31]:
# find_optimal_epochs()
# OPTIMAL EPOCHS: Around 10 (minimal differences)

In [59]:
model = create_model(num_features=X_train.shape[1], board_dir="relu,epochs=10")
fit_model(model, X_train, Y_train, X_test, Y_test, epoch=10)

Training Step: 4159  | total loss: 0.13946 | time: 19.665s
| Adam | epoch: 010 | loss: 0.13946 - acc: 0.9457 -- iter: 26560/26566
Training Step: 4160  | total loss: 0.14285 | time: 20.735s
| Adam | epoch: 010 | loss: 0.14285 - acc: 0.9449 | val_loss: 0.15912 - val_acc: 0.9448 -- iter: 26566/26566
--


In [60]:
model.save('model/tflearnlol.model')

INFO:tensorflow:/Users/borisyue/Desktop/lol-champion-picker/lol-champion-picker/tensorflow/model/tflearnlol.model is not in all_model_checkpoint_paths. Manually adding it.


In [62]:
model.load('model/tflearnlol.model')

INFO:tensorflow:Restoring parameters from /Users/borisyue/Desktop/lol-champion-picker/lol-champion-picker/tensorflow/model/tflearnlol.model


In [98]:
buckets = np.zeros(276)
buckets

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [99]:
buckets[59] = 1
buckets[57] = 1
buckets[62] = 1
buckets[97] = 1
buckets[117] = 1
buckets

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [100]:
buckets[138 + 67] = 1
buckets[138 + 87] = 1
buckets[138 + 1] = 1
buckets[138 + 46] = 1
buckets[138 + 48] = 1

In [101]:
buckets = np.array(buckets).reshape(1, 276)
buckets

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.

In [103]:
model.predict(buckets)

array([[ 0.98279852,  0.01720144]], dtype=float32)